In [ ]:
from pathlib import Path

# Define paths
PROJECT_ROOT = Path.cwd().resolve().parent.parent
DATA_DIR = PROJECT_ROOT / "data"
OUTPUTS_DIR = PROJECT_ROOT / "outputs" / "voxel_statistics"

OUTPUTS_DIR.mkdir(parents=True, exist_ok=True)

# Load data

In [3]:
import polars as pl
import pandas as pd

clinical_df = pl.read_csv(DATA_DIR / "clean" / "clinical_data.csv")

In [ ]:
cognition_domain_scores = [
    var
    for var in clinical_df.columns
    if var.startswith("cognition-") and var.endswith("domain_score")
]
covariates = [
    "stroke-total_infarct_volume"
]
palm_df = clinical_df[["sub_id"] + covariates + cognition_domain_scores]
palm_df = palm_df.with_columns(
    pl.col(cognition_domain_scores).cast(pl.UInt8)
)

# Voxel statistics preparations

In [ ]:
PALM_DIR = OUTPUTS_DIR / "palm"
PALM_DIR.mkdir(exist_ok=True, parents=True)

fLNM_path = DATA_DIR / "processed" / "fLNM_2mm_leaddbs"

for domain in cognition_domain_scores:

    iteration_df = palm_df[["sub_id"] + [domain] + covariates].drop_nulls()

    iteration_df = iteration_df.with_columns(
        (
            pl.lit(str(fLNM_path) + "/")
            + pl.lit("resid_")
            + pl.col("sub_id").cast(pl.Utf8)
            + pl.lit("_infarct_conn-GSP1000v2xFullSet_desc-T_funcmap.nii.gz")
        ).alias("image_path")
    )

    Path(PALM_DIR / f"fLNM_{domain}").mkdir(
        exist_ok=True, parents=True
    )
    iteration_df[[domain] + covariates].write_csv(
        PALM_DIR / f"fLNM_{domain}/design.csv",
        include_header=False,
    )
    pd.DataFrame([1] + [0] * len(covariates)).T.to_csv(
        PALM_DIR / f"fLNM_{domain}/contrast.csv",
        header=None,
        index=None,
    )
    pd.DataFrame(iteration_df["image_path"]).to_csv(
        PALM_DIR / f"fLNM_{domain}/input_paths.txt",
        header=None,
        index=None,
    )